# NCNC Google Play Store Crawling
- Contributed By: Joon Ha Cha 

Resources From 
- https://github.com/park-gb/playstore-review-crawler

# Set Ups

### Installations

In [5]:
# Copy this code in terminal (make sure pip3 and python3 are installed)
# pip3 install --upgrade pip, install python 3 for mac online
# Check with python3 --version, pip3 --version

# Copy the code below to terminal
# pip3 install beautifulsoup4 && pip3 install selenium && pip3 install pandas
# pip3 install tqdm && pip3 install requests

# 또한, Chrome driver를 따로 설치하셔야합니다


In [6]:
# Chrome driver 경로지정
chrome_driver = '../chromedriver'

# URL for app collection
URL = 'https://play.google.com/store/apps/details?id=com.ncncnative'


### Importing Packages

In [7]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options

## 스크롤 Function

구글 플레이 스토어의 리뷰는 무한 스크롤 형태입니다.

스크롤을 내리면 새로운 리뷰를 보여주는 형식이다.

모든 리뷰를 확인하려면 스크롤을 맨 아래까지 내려야하기때문에, 아래의 함수를 이용합니다

In [8]:
# 스크롤을 내리는 function (불러오기 포함, 최하단으로)
def scroll(model):
    try:        
        last_height = driver.execute_script("return arguments[0].scrollHeight", model)
        while True:
            # 아무 숫자 넣어도 됩니다. 약 0.5~2초 사이면 될듯 시간 아까우니
            pause = 1.5

            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", model)
            time.sleep(pause)

            # 끝까지 스크롤을 하기 위한 위치조정
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", model)
            time.sleep(pause)

            # 스크롤 높이 측정
            new_height = driver.execute_script("return arguments[0].scrollHeight", model)

            try:
                # '더보기' 버튼 경우
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

## 데이터 크롤링

여기서부터 크롬 드라이버 종료 탭까지는 한번에 run해야 에러가 안뜹니다.

In [9]:
# Setting Chrome Drivers 크롬 드라이버 미리 설치되어야 합니다
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome(chrome_driver)
driver.maximize_window()

# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인(2022-08-08 selenium 새버전으로 고침)
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))

# '리뷰 모두 보기' 버튼 클릭 (2022-08-08 xpath 재설정)
driver.find_element("xpath", all_review_button_xpath).click()
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 스크롤
model = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(model)

/var/folders/zh/bcgn50xs2hnctqtmqs93m5yw0000gn/T/ipykernel_2713/878062142.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)


In [10]:
# html parsing
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

In [11]:
# 크롬 드라이버 종료
driver.quit()

## 데이터 저장

In [12]:
# html 데이터 저장
with open("../dataset/data_html.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

## 저장된 데이터 전처리

In [13]:
# 리뷰 데이터 클래스 설정
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 필요한 Value들 설정
dataset = []
review_num = 0 
# 스크랩한 전체 리뷰정보 추출
for review in review_source:
    review_num+=1
    # 리뷰 등록일
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    # 닉네임
    user_name = review.find_all(class_ = 'X5PpBb')[0].text 
    # 평점 (여섯번째 단어가 평점 숫자)
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10]
    # 리뷰 (컨텐츠)
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

In [14]:
# date 전처리는 추후 analyze할때 추가로 정리해야함 (현재는 June 4, 2022 형식으로 저장되어있음)

In [15]:
df_crawl = pd.DataFrame(dataset)
df_crawl.to_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

## 리뷰 데이터 불러오기

In [16]:
# 저장한 리뷰 정보 불러오기
df_crawl = pd.read_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig')
df_crawl = df_crawl.drop(['Unnamed: 0'], axis = 1)
df_crawl

,id,date,rating,userName,content
0,1,2022년 8월 11일,5,김소연,잘쓰고 있는데요. 구매기한 다되가는거 알람울려 사용하는 푸쉬기능하고.. 사놓은 기프...
1,2,2022년 8월 12일,5,xx zz,앱 깔끔하고 빨라서 좋습니다! 기능 추가됐으면 하는 점이 2개 있습니다. 1. 판매...
2,3,2022년 1월 11일,5,SJ PD,지금까지 써 본 기프티콘 구매•판매 어플 중 단연 최고인듯 합니다. 앱 구성도 복잡...
3,4,2021년 3월 17일,5,김잔디,니콘내콘 아주 잘이용하고있어요. 다른 곳보다도 저렴한편이고 사용방법이 간단하여 참 ...
4,5,2020년 8월 11일,5,SH L,별5개 ☆☆☆☆☆ 별점이 올라가는 EVENT..! 비교하는 게 조금 마음에 걸릴 수...
...,...,...,...,...,...
835,836,2019년 11월 15일,5,Google 사용자,할인쿠폰 놓칠수없죠~ㅋ
836,837,2021년 5월 10일,5,윶윶,저렴하게 살 수 있어서 좋아요!!!!
837,838,2021년 6월 10일,1,진진,앱 접속이 계속 안돼요
838,839,2020년 10월 16일,5,박진성,니콘내콘 최고에요 무조건 강추


## 리뷰 데이터 분석 & 추가 전처리

In [17]:
# 전처리 위한 numpy 불러오기
import numpy as np

In [18]:
# 데이터 손실 방지
df = df_crawl.copy()

# 리뷰 기본정보
print('리뷰 크롤링 기간: '+min(df['date']) +' ~ ' + max(df['date']))
print('리뷰 수: ' + str(review_num) +'개')

# 전처리가 잘 되었는지 확인
print('유니크한 레이팅: ' + str(df['rating'].unique()))


리뷰 크롤링 기간: 2019년 10월 15일 ~ 2022년 8월 9일
리뷰 수: 840개
유니크한 레이팅: [5 4 3 1 2]


In [19]:
# Checking Null and Repeated usernames
print('유니크한 유저이름: ' + str(df['userName'].unique().shape[0]))

temp_df = df.groupby('userName').count() > 2
temp_df = temp_df.reset_index()
lst = []
for i in range(len(temp_df)):
    if temp_df['id'][i] == True:
        lst.append(temp_df['userName'][i])

for i in range(len(lst)):
    print('중복 닉네임 이름: ' + str(lst[i]) + ', ' + str(sum(df['userName'] == lst[i])) + '개')

print('NULL 이름값 : ' + str(sum(df['userName'].isna())))

유니크한 유저이름: 784
중복 닉네임 이름: Google 사용자, 51개
NULL 이름값 : 0


In [20]:
# 평점 카운트 비율
df.groupby('rating').count()['id']
rating_p = []

for i in range(5):
    rating_p.append(df.groupby('rating').count()['id'].iloc[i])

rating_p = np.round((rating_p/sum(rating_p)), 3) * 100

rating_p_dict = {'1': rating_p[0], '2': rating_p[1], '3': rating_p[2], '4': rating_p[3], '5': rating_p[4]}
rating_p_dict


{'1': 22.6, '2': 3.2, '3': 8.0, '4': 8.6, '5': 57.599999999999994}

## 크롤링된 리뷰 ML로 감성분석 (긍정,부정)

In [21]:
# regex용 패키지 pregex (python 3.9 이상부터 사용가능, pip3 install pregex)
from pregex.classes import AnyButWhitespace
from pregex.quantifiers import AtLeast
from pregex.operators import Either

In [22]:
# 필요한 패키지들 불러오기
import matplotlib.pyplot as plt
import seaborn as sns

# scikit-learn 패키지들 불러오기
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# Warning 무시하기
import warnings
# warnings.fiterwarnings('ignore')

from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran


### 학습을 위한 데이터 전처리

In [23]:
# 평점 5 --> 긍정 평점 1,2,3,4 --> 부정으로 분류하는 함수
def pos_neg(dataframe):
    lst = []
    for i in range(len(dataframe)): 
        # 1 = 긍정 , 0 = 부정
        if dataframe['rating'][i] == 5:
            lst.append(1)
        else:
            lst.append(0)
    dataframe['pos_neg'] = lst
    return dataframe

# 긍,부정이 포함된 새로운 데이터프레임
df_pos_neg = pos_neg(df)

# 한글 정규식을 이용하여 한글을 제외한 글자 제거
def clean_korean(content):
    korean = re.compile('[^ ㄱ-ㅣ가-힣]+')
    rev = korean.sub('', content)
    return rev

# 한글 함수 적용
df_pos_neg['content'] = df_pos_neg['content'].apply(lambda x: clean_korean(x))

# 학습에 불필요한 column들 제거
columns_to_drop = ['id', 'date', 'userName']
df_pos_neg = df_pos_neg.drop(columns = columns_to_drop, axis=1)
df_pos_neg


,rating,content,pos_neg
0,5,잘쓰고 있는데요 구매기한 다되가는거 알람울려 사용하는 푸쉬기능하고 사놓은 기프티콘 ...,1
1,5,앱 깔끔하고 빨라서 좋습니다 기능 추가됐으면 하는 점이 개 있습니다 판매시 일괄승...,1
2,5,지금까지 써 본 기프티콘 구매판매 어플 중 단연 최고인듯 합니다 앱 구성도 복잡하지...,1
3,5,니콘내콘 아주 잘이용하고있어요 다른 곳보다도 저렴한편이고 사용방법이 간단하여 참 좋...,1
4,5,별개 별점이 올라가는 비교하는 게 조금 마음에 걸릴 수도 있지만 발전을 위한 조...,1
...,...,...,...
835,5,할인쿠폰 놓칠수없죠ㅋ,1
836,5,저렴하게 살 수 있어서 좋아요,1
837,1,앱 접속이 계속 안돼요,0
838,5,니콘내콘 최고에요 무조건 강추,1


In [24]:
# 샘플들을 미리 확인해본 결과, 1,2,3,4점은 통산적으로 보완점에 대한 이야기가 많았음
# 5점은 칭찬이 많음 (가끔 추가해줬으면 하는 기능이야기도 함) (피드백 느낌)
df[df['rating'] == 4].sample(10)

df_pos_neg['pos_neg'].value_counts()

1    484
0    356
Name: pos_neg, dtype: int64

In [25]:
# Content의 평균 길이 
sum_length = 0

for i in range(len(df_pos_neg)):
    sum_length += len(df_pos_neg['content'][i])

sum_length/df_pos_neg.shape[0]

62.44642857142857

In [26]:
# Py-Hanspell (네이버 한글 맞춤법 검사기를 바탕으로 만든 패키지)로 문법 보정
from hanspell import spell_checker

# 예시
example = "내가 뭐 왜 않돼는거지"
print("haspell 적용 전: " + example)
check_example = spell_checker.check(example)
hanspell_example = check_example.checked
print("hanspell 적용 후: " + hanspell_example)

haspell 적용 전: 내가 뭐 왜 않돼는거지
hanspell 적용 후: 내가 뭐 왜 안되는 거지


In [27]:
# 리뷰 content에 전체 적용을 위한 함수
def hanspell_apply(df):
    lst = []
    for i in range(len(df['content'])):
        check = spell_checker.check(df['content'][i])
        hanspell_check = check.checked
        lst.append(hanspell_check)
    return lst

# 원래의 dataframe에 함수 적용 후 저장
new_column = hanspell_apply(df_pos_neg)
df_pos_neg['fixed_context'] = new_column
df_pos_neg.drop(columns = 'content', inplace = True)

In [28]:
df_pos_neg['fixed_context']

0      잘 쓰고 있는데요 구매 기한 다 돼가는 거 알람 울려 사용하는 푸시 기능하고 사놓은...
1      앱 깔끔하고 빨라서 좋습니다 기능 추가됐으면 하는 점이 개 있습니다  판매 시 일괄...
2      지금까지 써 본 기프티콘 구매 판매 어플 중 단연 최고인듯합니다 앱 구성도 복잡하지...
3      니콘 내 콘 아주 잘 이용하고 있어요 다른 곳보다도 저렴한 편이고 사용방법이 간단하...
4      별개  별점이 올라가는  비교하는 게 조금 마음에 걸릴 수도 있지만 발전을 위한 조...
                             ...                        
835                                        할인쿠폰 놓칠 수 없죠ㅋ
836                                     저렴하게 살 수 있어서 좋아요
837                                         앱 접속이 계속 안돼요
838                                   니콘 내 콘 최고예요 무조건 강추
839                           어째서 미성년자는 깊이 판매를 못 하는 건가요ㅜ
Name: fixed_context, Length: 840, dtype: object

In [29]:
# SOYNLP를 이용한 형태소 분석
# 비지도 학습으로 단어 토큰화 << 신조어, 비속어 등 구분이 가능하게 학습가능
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

# Initializing WordExtractor
extractor = WordExtractor()
extractor.train(df_pos_neg['fixed_context'])
extractor_score_table = extractor.extract()

training was done. used memory 0.554 Gby 0.547 Gb
all cohesion probabilities was computed. # words = 1339
all branching entropies was computed # words = 2283
all accessor variety was computed # words = 2283


여기서 SOYNLP의 응집 확율(Cohesion Probability)의 자세한 관련식 링크:
https://wikidocs.net/92961 

한글은 L토큰 + R토큰으로 분리하고 L토큰을 가져오는게 유리하다
하지만 신기하게도 두개 비교결과, max score tokenizer 가 좀 더 정확하게 유추하여서 max score tokenizer 채택

In [30]:
# Further Tokenizing
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.tokenizer import LTokenizer
from collections import Counter

# LTokenizer
# scores = {word:score.cohesion_forward for word, score in extractor_score_table.items()}
# tokenizer = LTokenizer(scores = scores)
# tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 철결하자", flatten = False)

# MaxScoreTokenizer
scores = {word:score.cohesion_forward for word, score in extractor_score_table.items()}
maxscore_tokenizer = MaxScoreTokenizer(scores = scores)

# Tokenizer 데이터에 적용
lst = []
for i in range(len(df_pos_neg)):
    lst.append(maxscore_tokenizer.tokenize(df_pos_neg['fixed_context'][i]))

# 추출한 리스트 한개로 합치기 (21000개)
lst_words = [element for innerList in lst for element in innerList]

# 추출한 데이터 전처리 (한글자, 불용어 제거) 16522개
no_singular_word = [x for x in lst_words if len(x) > 1]

# 불용어 제거 (15000개)
with open('stop_words.txt', 'r') as f:
    list_file = f.readlines()
stop_lst = []
for i in range(len(list_file)):
    temp_str = list_file[i].replace("\n","")
    stop_lst.append(temp_str) 
remove_stopwords = [x for x in no_singular_word if x not in stop_lst]

# 단어별 빈도 확인
counter = Counter(remove_stopwords).most_common()

# 빈도가 3 이하면 제거 (총 704개)
min_count = 3
min_count_stopwords = [(x,y) for x,y in counter if y > 3]

In [1]:
1+1

2

In [31]:
# 위에 진행된 과정을 한개의 함수로 압축시키기
def cleaning_context(fixed_content, stop_lst):
    # 한글 문법 정리
    words = clean_korean(fixed_content)

    # 형태소 추출
    extractor = WordExtractor()
    extractor.train(df_pos_neg['fixed_context'])
    extractor_score_table = extractor.extract()

    # MaxScoreTokenizer
    scores = {word:score.cohesion_forward for word, score in extractor_score_table.items()}
    maxscore_tokenizer = MaxScoreTokenizer(scores = scores)

    # Tokenizer 데이터에 적용
    lst = []    
    for i in range(len(df_pos_neg)):
        lst.append(maxscore_tokenizer.tokenize(df_pos_neg['fixed_context'][i]))
        
    # 추출한 리스트 한개로 합치기
    words = [element for innerList in lst for element in innerList]

     # 추출한 데이터 전처리 (한글자, 불용어 제거) 
    words = [x for x in lst_words if len(x) > 1]

    # 불용어 제거 
    words = [x for x in words if x not in stop_lst]

    # 단어별 빈도 확인
    counter = Counter(remove_stopwords).most_common()

    # 빈도가 3 이하면 제거 (총 704개)
    min_count = 3
    words = [(x,y) for x,y in counter if y > 3]

    return words

def applying_stopwords(content, stopwords_lst):
    lst = ''
    for i in range(len(stopwords_lst)):
        if stopwords_lst[i][0] in content:
            lst = lst + ' ' + stopwords_lst[i][0]
    return lst

In [56]:
# 핵심어들 빈도 count
word_list = cleaning_context(" ".join(df_pos_neg['fixed_context'].tolist()), stop_lst)
word_list

training was done. used memory 1.606 Gby 1.602 Gb
all cohesion probabilities was computed. # words = 1339
all branching entropies was computed # words = 2283
all accessor variety was computed # words = 2283


[('사용', 322),
 ('기프티콘', 251),
 ('판매', 228),
 ('구매', 211),
 ('좋아요', 197),
 ('쿠폰', 168),
 ('니콘', 142),
 ('어플', 99),
 ('결제', 95),
 ('저렴하게', 83),
 ('환불', 80),
 ('할인', 73),
 ('이용', 63),
 ('필요', 62),
 ('가격', 55),
 ('상품', 53),
 ('싸게', 53),
 ('확인', 53),
 ('인증', 53),
 ('카드', 50),
 ('계속', 48),
 ('이미', 47),
 ('있습니다', 45),
 ('자주', 45),
 ('다시', 45),
 ('좋은', 44),
 ('등록', 44),
 ('많이', 43),
 ('저렴', 41),
 ('고객센터', 41),
 ('문의', 40),
 ('좋네요', 40),
 ('상담', 40),
 ('없는', 38),
 ('좋겠어요', 36),
 ('안되', 34),
 ('ㅠㅠ', 34),
 ('충전', 33),
 ('같은', 32),
 ('처리', 32),
 ('금액', 32),
 ('가능', 31),
 ('제가', 31),
 ('하는데', 31),
 ('사기', 31),
 ('있고', 30),
 ('불편', 30),
 ('진짜', 29),
 ('좋습니다', 28),
 ('만족', 28),
 ('선물', 28),
 ('오류', 28),
 ('처음', 27),
 ('편리하', 27),
 ('없고', 26),
 ('매입', 25),
 ('생각', 25),
 ('된다', 25),
 ('해주세요', 25),
 ('이용하고', 24),
 ('아주', 23),
 ('답변', 23),
 ('이용하', 23),
 ('정산', 23),
 ('편하고', 23),
 ('불가', 23),
 ('앱이', 23),
 ('리뷰', 22),
 ('완료', 21),
 ('버튼', 21),
 ('있으면', 20),
 ('점이', 20),
 ('최고', 20),
 ('추천', 20),
 ('없어서',

In [33]:
# Applying it to the original data
result_lst = []
for i in range(len(df)):
    result_lst.append(applying_stopwords(df['content'][i], word_list))

## Keybert NLP Method

Null Hypothesis: 리뷰데이터에서 keybert를 이용한 키워드 추출을 통해 추출된 핵심어와 감성분석이 리뷰 원문의 의미와 일치하도록 하는 리뷰 분석 파이프라인을 만들어 80%이상의 일치율이 없다

Alternative Hypothesis: 리뷰데이터에서 keybert를 이용한 키워드 추출을 통해 추출된 핵심어와 감성분석이 리뷰 원문의 의미와 일치하도록 하는 리뷰 분석 파이프라인을 만들지만 일치율이 있다

가설이 맞을 경우, 핵심단어와 감성분석만을 보아 전체적인 문제점과 의미 파악 가능. 또한, 위에 구한 긍부정을 이용해 현재 고객이 생각하는 ncnc의 장점과 단점 구별가능.

The input document is embedded using a pre-trained BERT model. You can pick any BERT model your want from transformers. This turns a chunk of text into a fixed-size vector that is meant the represent the semantic aspect of the document.


The BERT model used in this experiment --> istilbert-base-nli-mean-tokens
(Text is embedding in vector space such that similar text is close and can efficiently be found using cosine similarity)

Cosine similarity measures the similarity between two vectors of an inner product space. 

MMR(maximal marginal relevance) (adds diversity) 또한 포함이 되어있음



In [57]:
from keybert import KeyBERT
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [41]:
df_pos_neg.head()

,rating,pos_neg,fixed_context
0,5,1,잘 쓰고 있는데요 구매 기한 다 돼가는 거 알람 울려 사용하는 푸시 기능하고 사놓은...
1,5,1,앱 깔끔하고 빨라서 좋습니다 기능 추가됐으면 하는 점이 개 있습니다 판매 시 일괄...
2,5,1,지금까지 써 본 기프티콘 구매 판매 어플 중 단연 최고인듯합니다 앱 구성도 복잡하지...
3,5,1,니콘 내 콘 아주 잘 이용하고 있어요 다른 곳보다도 저렴한 편이고 사용방법이 간단하...
4,5,1,별개 별점이 올라가는 비교하는 게 조금 마음에 걸릴 수도 있지만 발전을 위한 조...


In [59]:
# Bert를 이용한 키워드 추출 함수
# State 1 = positive, 0 = negative
def BERT(df, state):
    df_2 = df[df["pos_neg"] == state]
    review_array = df_2['fixed_context'].to_numpy()

    bow = []
    extractor = KeyBERT('distilbert-base-nli-mean-tokens')
    
    for j in range(len(review_array)):
        keywords = extractor.extract_keywords(review_array[j], diversity = 0.7)
        bow.append(keywords)
    
    final_bow = []
    for i in range(0, len(bow)):
        for j in range(len(bow[i])):
            final_bow.append(bow[i][j])
    
    keyword = pd.DataFrame(final_bow, columns=['keyword', 'weight'])
    print(keyword.groupby('keyword').agg('sum').sort_values('weight', ascending=False).head(20))

In [60]:
# 긍정 리뷰의 키워드
BERT(df_pos_neg, 1)

          weight
keyword         
저렴하게     60.1867
기프티콘     57.2137
기프티콘을    34.8233
좋아요      27.4399
사용하고     25.7502
너무       25.4942
편하고      16.9821
구매할      16.1158
이용하고     14.6237
싸게       14.5225
니콘       12.6678
유용하게     11.4214
자주       11.0927
필요한      11.0490
구매해서     10.8489
저렴한      10.7027
가능해서      9.7069
편리하고      9.4581
감사합니다     9.3414
정말        8.7923


In [61]:
# 부정 리뷰의 키워드 
BERT(df_pos_neg, 0)

          weight
keyword         
기프티콘     18.0528
기프티콘을    11.4422
저렴하게     10.2858
고객센터      9.4312
갑자기       7.9682
쿠폰        7.4242
기프티콘이     6.5245
하는데       5.9342
사용        5.3384
쿠폰이라고     5.0412
불편해요      4.9545
결제가       4.8835
환불도       4.8359
사용하려고     4.8157
핸드폰       4.8081
실행이       4.7470
쿠폰을       4.6643
로그인이      4.3040
사용하고      4.1087
고객센터는     4.0881


In [67]:
# 현 데이터 기준 긍/부정 리뷰 비율
pos = df_pos_neg[df_pos_neg['pos_neg'] == 1].shape[0]
neg = df_pos_neg[df_pos_neg['pos_neg'] == 0]
pos/neg
# pos 1.36: neg 1

1.3595505617977528

In [70]:
any(x in df_pos_neg["fixed_context"] for x in "고객센터")

False

In [96]:
# 단어를 대체해서 특정 단어가 들어있는 context 읽어보기
neg = df_pos_neg[df_pos_neg['pos_neg'] == 0]

lst = []
for i in range(len(neg)):
    if "갑자기" in neg['fixed_context'].iloc[i]:
        lst.append(neg['fixed_context'].iloc[i])
lst

['잘 되던 앱이 어제부터 갑자기 먹통이 되네요 캐시를 삭제해 보고 지웠다가 다시 설치해 봐도 그때만 되고 안됩니다 오늘은 상담원과 톡으로 상담을 했는데 제가 했던 방법으로 앱 삭제 캐시 삭제를 권고하네요 관리자님 조치해 주세요ㅜㅜ',
 '왜 로그인도 안 되고 고객센터도 안되고 무한 반복이에요 지금 앱 하나도 안돼요 갑자기 로그아웃됐어요 뭐 하시는 거예요',
 '그동안 잘 사용하고 있었는데 사용된 기프티콘인지 개가 갑자기 오류로 뜨면서 결제가 안되더니 가게에서 그냥 나오고 폰으로 환불 신청하니까 달 가까이 연락이 없네요',
 '카드로 결제하고 싶은데요 예전이 카드 등록을 했더니 갑자기 카트 미등록이 되어있더라고요 그래서 다시 등록할라 더니 기인증이라고ㅠㅠㅠ 등록 취소 이런 건 없나요',
 '잘 쓰고 있었는데 요즘 갑자기 결제가 안되고 대기화면만 몇 분 유지되길래 지웠어요',
 '쿠폰 결제했는데 갑자기 결제가 진행되지 않았다네요 돈은 결제됐는데 쿠폰은 없고',
 '다 좋은데 갑자기 구매 오류라 하면서 구매가 안돼서 화나서 지움 그리고 다시 깔았는데 카드 등록도 안됨',
 '아니 갑자기 결제가 시간이 지나도 안 되는 현상이 발생하는데 왜 이런 거죠 지웠다 깔아도 그러고',
 '어플 잘 쓰고 있는데 갑자기 왜 구 매 후에 무한 로딩만 되죠  결제는 이미 됐는데 ㅡㅡ 확인해주세요',
 '로그인 왜 안 되나 갑자기 왜 이래요 저번에 판매한 거 출금 신청도 안 했는데 아 짜증 나 로그인해 놨었는데 로그아웃됐네',
 '업데이트했는데 왜 안 열리고 자꾸 꺼지나요 갑자기 앱에 들어갈 수도 없네요',
 '며칠 전부터 갑자기 어플이 안 켜져요',
 '갑자기 안 열리네요 돈도 충전돼있는데 무슨 일이죠']